## Imports

Im [rlHelloWorld](experiments/rlHelloWorld.ipynb) wurde ein anderes reinforcement learning framework genutzt, welches glaube ich eng mit tensorflow zusammenarbeitet. Die Libraries unten sind aber die originalen von Tensorflow aus dem professionellen Cart Pole Tutorial von Tensorflow [Tutorial](https://github.com/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb)

In [3]:
import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

ImportError: This version of TensorFlow Probability requires TensorFlow version >= 2.8; Detected an installation of version 2.5.0. Please upgrade TensorFlow to proceed.

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam